In [221]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import math

import logging

In [222]:
def read_contents_file(file_path, encoding_scheme='cp1252', read_lines=False):
    inputfile = open(file_path, 'r', encoding=encoding_scheme)
    if read_lines:
        return inputfile.readlines()
    else:
        return inputfile.read()

In [223]:
#Write file to local directory
def write_file_local(path,file_data,is_list=False):
    
    with open(path, 'w') as file:
        if is_list:
            file_string = '\n'.join(file_data)
        else:
            file_string = file_data
        file.write(file_string)

In [224]:
#get the DDLs for the tables from teradata 
def cron_expression_generator():
    dt1 = datetime.now()
    
    try:
        df = pd.read_csv("dag_id_timings.csv", index_col=None)
        file_contents = []
        for index, row in df.iterrows():

            dag_id = str(row['Dag_ID']).strip().lower()
            dag_time = str(row['Time']).strip().lower()
            dag_frequency = str(row['Frequency']).strip().lower()
            dag_day_of_month = int(row['Day_of_Month']) if not math.isnan(row['Day_of_Month']) else np.nan
            dag_day_of_week = int(row['Day_of_Week']) if not math.isnan(row['Day_of_Week']) else np.nan

            hour_of_day = dag_time.split(":")[0] if len(dag_time.split(":")) > 1 else dag_time
            if hour_of_day != "*":
                hour_of_day = int(hour_of_day)

            minute_of_day = dag_time.split(":")[1] if len(dag_time.split(":")) > 1 else '*'
            if minute_of_day != "*":
                minute_of_day = int(minute_of_day)

            if dag_frequency == "monthly":
                day_of_month = dag_day_of_month
            else:
                day_of_month = "*"
            print(day_of_month)

            if dag_frequency == "weekly":
                day_of_week = dag_day_of_week
            else:
                day_of_week = "*"

            cron_expression = f"{minute_of_day} {hour_of_day} {day_of_month} * {day_of_week}"

            file_contents.append(f"{dag_id},{cron_expression}")
        
        write_file_local("dag_id_cron_expressions.csv", file_contents, True)

    except Exception as e1:
        print(e1)
        # print("Database : {}, Table : {}".format(database_name,table_name))
        pass
    
    dt2 = datetime.now()
    print(dt2-dt1)

    print("Record Count : ", len(df))

In [225]:
print("Begin of Processing")

cron_expression_generator()

print("End of Processing")

Begin of Processing
0:00:00.006995
Record Count :  6
End of Processing
